# Demonstration of Posit Orbital

[Introduction to Orbital](https://posit.co/blog/introducing-orbital-for-scikit-learn-pipelines/)

[Orbital Repository](https://github.com/posit-dev/orbital)

We are going to do a Logistic Regression model. The data is what was used at our Frequent Attenders Hackathon, and the processing, pipeline and model are based on what Paul J has done for his [speedrun demo](https://github.com/NHS-South-Central-and-West/frequent-attenders/tree/main) 

## Set up our warehouse connection

This is getting the data we used for the hackathon directly from the data warehouse.

**Note:** the `params.py` has been .gitignored so that the confidential connection details don't get published online.

In [ ]:
import pandas as pd
from utils.warehouse_connection import warehouse_connection
from utils import params

df = warehouse_connection(f'SELECT * FROM {params.TABLE_NAME}')

df.info()

## Create train and test datasets

In [ ]:
from utils.split_data import tt_split

train_df, test_df = tt_split(df)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

## Preprocessing

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
feats = ['Organisation_Code_Provider', 'Age_At_Arrival',
       'Index_Of_Multiple_Deprivation', 'Stated_Gender',
       'Arrival_Mode_Desc', 'Attendance_Category',
       'Long_Term_Condition_Count_Number', 'GP_Practice_Code',
       'Care_Home_Status', 
       'Living_Alone',
       'Disability_Count_Number',
       'Segmentation_Bridges_To_Health',]

target = ['frequent_attender']

In [ ]:
feats + target

In [ ]:
train_df = train_df.dropna(subset=feats)

In [ ]:
train_df['Index_Of_Multiple_Deprivation'] = train_df['Index_Of_Multiple_Deprivation'].astype(int)

In [ ]:
X = train_df[feats]
y = train_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Create the model pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np

cat_feats = ['Organisation_Code_Provider', 'Stated_Gender', 'Arrival_Mode_Desc', 
            'GP_Practice_Code', 'Living_Alone', 'Attendance_Category', 'Care_Home_Status']
num_feats = ['Age_At_Arrival', 'Index_Of_Multiple_Deprivation', 'Long_Term_Condition_Count_Number', 
            'Segmentation_Bridges_To_Health', 'Disability_Count_Number']

categorical_transformer = Pipeline(steps=[("encoder", OneHotEncoder(handle_unknown='ignore'))])
numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

col_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_feats),
        ("cat", categorical_transformer, cat_feats),
    ]
)

preprocessor = Pipeline(steps=[("col_transformer", col_transformer)])

clf = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("regressor", LogisticRegression(max_iter=1000))]
)

clf.fit(X_train, np.ravel(y_train))

## Create an Orbital pipeline

In [ ]:
import orbital

We need the X column names and the data types to go into the Orbital pipeline.

In [ ]:
X.info()

Would be nice to have a way to do this without having to write them all out again, but `orbital.types.guess_datatypes(X)` didn't work.

In [ ]:
orbital_pipeline = orbital.parse_pipeline(
    pipeline=clf,
    features={
        'Organisation_Code_Provider': orbital.types.StringColumnType(),
        'Age_At_Arrival': orbital.types.Int64ColumnType(),
        'Index_Of_Multiple_Deprivation': orbital.types.Int64ColumnType(),
        'Stated_Gender': orbital.types.StringColumnType(),
        'Arrival_Mode_Desc': orbital.types.StringColumnType(),
        'Attendance_Category': orbital.types.StringColumnType(),
        'Long_Term_Condition_Count_Number': orbital.types.FloatColumnType(),
        'GP_Practice_Code': orbital.types.StringColumnType(),
        'Care_Home_Status': orbital.types.StringColumnType(),
        'Living_Alone': orbital.types.StringColumnType(),
        'Disability_Count_Number': orbital.types.FloatColumnType(),
        'Segmentation_Bridges_To_Health': orbital.types.FloatColumnType(),
    }
)

In [ ]:
print(orbital_pipeline)

## Generate SQL

Get the list of SQL dialects available, since this is a required argument for generating the SQL. Bung it into a DataFrame to make it more easily browsable with Data Wrangler.

Ours is listed as "TSQL".

In [ ]:
import sqlglot
dialects = pd.DataFrame(sqlglot.dialects.DIALECTS)
dialects

# if you don't have Data Wrangler, it's the last one, so run dialects.tail()

Having said that, I tried "TSQL" and it didn't work, returning an Ibis-related error. Looking at the [docs](https://ibis-project.org/backends/mssql), the dialect of SQL we need is "mssql".

In [ ]:
from utils import params

sql = orbital.export_sql(
    table_name=params.JUST_THE_TABLE,  # sqlglot just wants the table name as a string, not [d].[s].[t] format.
    pipeline=orbital_pipeline,
    dialect='mssql',
    optimize=False # this prevents sqlglot from trying to produce aliases for the pipeline, which trips it up when using mssql.
)

print(sql)

### Making the MSSQL produced by Orbital compatible with our 2016 SQL Server.

Orbital produces SQL that uses functions that have been introduced more recently than 2016, so some of the functions are not available. For example, Orbital uses the function `GREATEST`, which only became supported by SQL Server from 2022.

In [ ]:
from utils.sql_compatibility import adapt_sql

In [ ]:
adapted_sql = adapt_sql(sql)

In [ ]:
print(adapted_sql)

Let's try running the adapted SQL. On the first attempt, this took about two minutes to run.

In [ ]:
results = warehouse_connection(adapted_sql)

results.head()